## project luther

Nathaniel Kelley

01/25/18

## imports and driver setup

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import os
import re
from selenium.webdriver.chrome.options import Options


chromedriver = "/home/nate/Bench/chromedriver"
os.environ["webdriver.chrome.driver"] = chromedriver

In [ ]:
options = Options() 
options.add_argument("--start-maximized") 
driver=webdriver.Chrome(chromedriver, chrome_options=options) 

# scrape for individual page

In [ ]:
def house_info(driver):
    
    '''
    FOR: https://www.redfin.com
    
    call once on the webpage for an individual house
    
    variables scraped include: 
            address, home price, beds, baths, house size by sqft, lot size by sqft, 
            year built, garage type, yearly property tax, number of nearby buslines
            
    stored in a dictionary with address as the key, so any address scraped twice
    won't be duplicated
    '''
    

    address = driver.find_element_by_xpath('//*[@class="street-address"]').text    
    csz = driver.find_element_by_xpath('//*[@class="citystatezip"]').text    
    full_address = address + ', ' + csz


    home_price = driver.find_element_by_xpath('//*[@class="info-block price"]').text
    home_price = float(home_price.replace('$', '').replace(',', '').replace('\nPrice', ''))


    beds = driver.find_element_by_xpath('//*[@data-rf-test-id="abp-beds"]').text
    try:
        beds = float(beds.replace('\n', ' ').split(' ')[0])
    except:
        beds = float(0)
        

    baths = driver.find_element_by_xpath('//*[@data-rf-test-id="abp-baths"]').text
    try:
        baths = float(baths.replace('\n', ' ').split(' ')[0])
    except:
        baths = float(0)
        
    
    house_sqft = driver.find_element_by_xpath('//*[@class="info-block sqft"]').text
    try:
        house_sqft = float(house_sqft.split(' ')[0].replace(',', ''))
    except:
        house_sqft = float(0)
        
        
    key_deets = driver.find_element_by_xpath('//*[@class="keyDetailsList"]').text
    key_deets = key_deets.split('Built')[1].split('\n')
    
    
    year_built = float(key_deets[1])
    
    
    try:
        lot_sqft = float(key_deets[3].split(' ')[0].replace(',', ''))
    except:
        lot_sqft = float(0)

       
    amenities = driver.find_element_by_xpath('//*[@class="amenities-container"]').text
    amenities = amenities.replace('\n', ' ')


    garage_find = re.compile('\w*tached G')
    garage_type = re.findall(garage_find, amenities)
    try:
        garage_type = str(garage_type[0]).lower().split(' ')[0]
    except:
        garage_type = 'none'

        
    try:
        try:    
            tax_find = re.search(r'Taxes:(.*?)Tax', amenities).group(1)
            yearly_tax = float(tax_find.replace(' ', '').replace('$', '').replace(',', ''))
        except:
            monthly_tax = driver.find_element_by_xpath('//*[@class="MortgageCalculatorSummary"]').text
            monthly_tax = monthly_tax.replace('\n', ' ')
            monthly_tax = re.search(r'Taxes(.*?)H', monthly_tax).group(1)
            monthly_tax = float(monthly_tax.strip().replace('$', '').replace(',', ''))
            yearly_tax = 12*monthly_tax
    except:
        yearly_tax = float(0)

        
    try:
        bus_find = re.search(r'Route Number: (.*)Listing', amenities).group(1)
        bus_list = bus_find.replace(',', ' ').split()
        num_busline = len(bus_list)
    except:
        num_busline = 0  
    
    

    
    stats = [full_address, home_price, beds, baths, \
            house_sqft, lot_sqft, year_built, \
            garage_type, yearly_tax, num_busline]
    
    return {stats[0]: stats[1:]}

# scraping the whole page

In [ ]:
def page_pull(driver):
    
    '''
    for page one
    
    open house, scrape, close, repeat
    
    close window after house 20 (max)
    '''

    for i in range(20):

        try:
            search = driver.find_element_by_xpath('//*[@id="MapHomeCard_{}"]/div/div[2]/div[3]/a/button'.format(i))
        except:
            search = driver.find_element_by_xpath('//*[@id="MapHomeCard_{}"]/div/div[2]/div[4]/a/button'.format(i))

        search.click()
        time.sleep(1);

        driver.switch_to_window(driver.window_handles[1]);

        house_pull.update(house_info(driver));
        time.sleep(1);


        driver.close();

        driver.switch_to_window(driver.window_handles[0]);

In [ ]:
def page_over_pull(driver):
    
    '''
    for page i in (2, n)
    
    open house, scrape, close, repeat
    
    close window after house 20*i (max)
    '''

    for i in range(20*(j-1),20*j):

        try:
            search = driver.find_element_by_xpath('//*[@id="MapHomeCard_{}"]/div/div[2]/div[3]/a/button'.format(i))
        except:
            search = driver.find_element_by_xpath('//*[@id="MapHomeCard_{}"]/div/div[2]/div[4]/a/button'.format(i))

        search.click()
        time.sleep(1);

        driver.switch_to_window(driver.window_handles[1]);

        house_pull.update(house_info(driver));
        time.sleep(1);


        driver.close();

        driver.switch_to_window(driver.window_handles[0]);

# scraping all the pages

In [ ]:
# initialize driver, something weird with starting maximized

options = Options() 
options.add_argument("--start-maximized") 
driver=webdriver.Chrome(chromedriver, chrome_options=options);

In [ ]:
# scrape page 1
# initially not included in whole loop because not 'https://...type=house/page-1'
# left on its own for page 1 update-scrapes

page_pull(driver);

driver.quit();

In [ ]:
# scrape pages 2 thru n
# can identify n, or just plug in high number and let it error off
# recommend checking n to avoid extra dummy houses at end


# Seattle:
# https://www.redfin.com/city/16163/WA/Seattle/filter/sort=lo-days,property-type=house/page-{}


n = 
city = ''
house_pull = {}

for j in range(2, n):
    driver.get(city.format(j));
    time.sleep(1);
    page_over_pull(driver);
    
driver.quit();

In [ ]:
# file names:
# Seattle: sea_raw.pkl


import pickle

pickle_out = open("  .pkl","wb")
pickle.dump(house_pull, pickle_out)
pickle_out.close()